#### CRISP

## Business Understanding

- There are 2 datasets `train_data.csv` and `test_data.csv`
- The `contest-tmp2m-14d__tmp2m` is the mean `(tmax+tmin / 2)` temperature and is to be predicted for the test data
- Latitude and longitude are anonymized so latitude information cannot be used for temperature prediction
- `startdate` indicates the start of a 14 day period
- The data provided is between **2014** and **2016**, therefore the affect of **El Nino** is to be considered
- `nmme` forecast values and other forecast values will not be part of the feature set used for this model
- The 2010 data for geopotential, wind, etc. will also be discarded for this model
- The 2010 data for sea surface temperature will be however used

NOTE: *There are inferences below some of the data analysis/visualization which dictates the next set of data transformations*

## Data Analysis

#### Import libraries and set Universal params

In [ ]:
# Import main libraries for data analysis and modelling
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, chi2, f_regression, f_classif, mutual_info_classif, SelectFromModel, RFE
from sklearn.linear_model import LinearRegression

import seaborn as sns
from matplotlib import pyplot as plt
from mpl_toolkits import mplot3d
import plotly.express as px

# Import additional helper libraries
import os
import datetime as dt
from IPython.display import display
# import math
# from math import radians, cos, sin, asin, sqrt
# import itertools
from shapely.geometry import Point
from shapely import wkt
import geopandas as gpd
from geopandas import GeoDataFrame


In [ ]:
pd.set_option("display.max.columns", None)

#### Set paths and create dataframes

In [ ]:
# Define the filepath

data_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + '/data/'

train_csv = data_dir + 'train_data.csv'
test_csv = data_dir + 'test_data.csv'

print(train_csv)
print(test_csv)

In [ ]:
# Load the training data set
train_df_raw = pd.read_csv(train_csv)

# Load the test data set
test_df_raw = pd.read_csv(test_csv)

#### Initial Analysis

In [ ]:
# Display primary observations
display(train_df_raw.info())
display(train_df_raw.head())
display(train_df_raw.tail())
display(train_df_raw.describe())

In [ ]:
with open('train_columns.txt', 'w', encoding='utf-8') as f:
    for col in train_df_raw.columns:
        f.write(f'{col},{train_df_raw.dtypes[col]},{len(train_df_raw[col].unique())}\n')

with open('train_df_info.txt', 'w', encoding='utf-8') as f:
    train_df_raw.info(verbose=True, buf=f)

- `startdate` is an object and needs to be converted to `datetime` and later to `ordinal` Int type for usability
- `climateregions__climateregion` is an object and needs to be converted to string type for usability

In [ ]:
# Find the target column
target_feature = train_df_raw.columns.difference(test_df_raw.columns)[0]
print(f'The target feature for prediction is {target_feature}')

In [ ]:
# Find any column with empty/null values

null_features = train_df_raw.columns[train_df_raw.isnull().any()] 
nan_features = train_df_raw.columns[train_df_raw.isna().any()]

print(f'Columns with null vaules in Training data are {train_df_raw.columns[train_df_raw.isnull().any()]}')
print(f'Columns with null vaules in Training data are {train_df_raw.columns[train_df_raw.isna().any()]}')

def get_null_percentage(df):
    null_counts = df.isnull().sum()
    null_percentage = null_counts / df.shape[0] * 100
    return {'null_counts': null_counts, 'null_percentage': null_percentage}

for nf in nan_features:
    print(nf,get_null_percentage(train_df_raw[nf]))

- Use **Imputer** to populate the `NaN` data
- Null percentage is small so could be imputed using mean/mode
- The features having `null/NaN` value are all prediction data and hence could be ignored

In [ ]:
# Check unique locations
print('Unique locations in train data ',train_df_raw.groupby(['lat','lon']).ngroup().nunique())
print('Unique locations in test data ',test_df_raw.groupby(['lat','lon']).ngroup().nunique()) 
print('Unique locations in combined data ',pd.concat([train_df_raw,test_df_raw], axis=0).groupby(['lat','lon']).ngroup().nunique())

Combined dataframe gives more unique locations than either train or test data. Check precision of location data to determine practicality

In [ ]:
# Get current precision of latitude and longitude
precision = train_df_raw[['lat','lon']].applymap(lambda x: len(str(x).split('.')[1]))

print(f'Current precision of latitude in training data is {precision.lat.max()}')
print(f'Current precision of longitude in training data is {precision.lon.max()}')

precision = test_df_raw[['lat','lon']].applymap(lambda x: len(str(x).split('.')[1]))

print(f'Current precision of latitude in test data is {precision.lat.max()}')
print(f'Current precision of longitude in test data is {precision.lon.max()}')

- Precision 16 is too high for practical purpose. This indicates a computer or calculator was used and that no attention was paid to the fact that the extra decimals are useless.
- The ninth decimal place is worth up to 110 microns. So, this is getting into the range of microscopy. 
- For almost any conceivable application with earth positions, this is overkill and will be more precise than the accuracy of any surveying device.
- Decision is to reduce precision to 6 decimal places

In [ ]:
# This will simply check whether a column is sorted. This is done as is_monotonic is deprecated
def check_sort(df,col):
    if df[col].is_monotonic_increasing or df[col].is_monotonic_decreasing:
        return True
    else:
        return False

In [ ]:
# Check the current sorting order on important columns
print('Sorted by index = ', check_sort(train_df_raw,'index'))
print('Sorted by latitude = ', check_sort(train_df_raw,'lat'))
print('Sorted by longitude = ', check_sort(train_df_raw,'lon'))
print('Sorted by date = ', check_sort(train_df_raw,'startdate'))

A useful sorting would be to sort by location (combined latitude and longitude) and then by date

##### Raw Data Visualizations

In [ ]:
# Check initial locations on a map to understand the geography of train data
location_df = train_df_raw[['lat','lon']].drop_duplicates().copy()
location_df.head()

geometry = [Point(xy) for xy in zip(location_df['lat'], location_df['lon'])]
gdf = GeoDataFrame(location_df, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=1);


In [ ]:
# Check initial locations on a map to understand the geography of test data
location_df = test_df_raw[['lat','lon']].drop_duplicates().copy()
location_df.head()

geometry = [Point(xy) for xy in zip(location_df['lat'], location_df['lon'])]
gdf = GeoDataFrame(location_df, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='o', color='red', markersize=1);


- Both train and test data are supposedly at the same location.
- `Location` is anonymized therefore it could be either omitted or converted to categorical feature

In [ ]:
# Visualize the spread of time in train and test data
time_df_train = pd.DataFrame()
time_df_train['startdate'] = pd.to_datetime(train_df_raw['startdate'], format='%m/%d/%y')

time_df_test = pd.DataFrame()
time_df_test['startdate'] = pd.to_datetime(test_df_raw['startdate'], format='%m/%d/%y')

fig, ax = plt.subplots(1,1, figsize=(16,2))
ax.set_title('Time Series Data')

sns.scatterplot(data = time_df_train, x = 'startdate', y = 1, marker='o', linewidth=0, label = 'train')
sns.scatterplot(data = time_df_test, x = 'startdate', y = 1, marker='o', linewidth=0, label = 'test')

ax.set_xlim([time_df_train['startdate'].iloc[0], time_df_test['startdate'].iloc[-1]])
plt.show()

time_df_train.groupby([time_df_train['startdate'].dt.year, time_df_train['startdate'].dt.month]).count().plot(kind='bar')
time_df_test.groupby([time_df_test['startdate'].dt.year, time_df_test['startdate'].dt.month]).count().plot(kind='bar')

In [ ]:
# Visualize target temperature 
temp_df = train_df_raw[['lat','lon','startdate','contest-tmp2m-14d__tmp2m']].copy()
temp_df['startdate'] = pd.to_datetime(temp_df['startdate'], format='%m/%d/%y')
temp_df['location'] = [Point(xy) for xy in zip(temp_df['lat'], temp_df['lon'])] 
temp_df['location_str'] = temp_df['location'].apply(lambda x: wkt.dumps(x)) # Testing Point geometry to String for use in ML training

temp_df = temp_df.pivot(index='startdate', columns='location_str', values='contest-tmp2m-14d__tmp2m')
temp_df.head()
temp_df.plot(legend=False)

- The temperature spread is even and reasoanable across the year matching the northern hemisphere seasonal temperature variation
- The temperature spread by location indicates the locations to be spread across a large geographical area and multiple climatic regions

#### Initial Data Transformations
Needed for the data to be properly visualized. E.g. Convert startdate from mm/dd/yy to ISO format; Sorting by location and date; Combine the latitude and longitude to create location; Reduce the precision of latitude and longitude to 6 to omit superfluous locations

In [ ]:
# Temporary functions to test

In [ ]:
# create new copies of the dataframes for further operations
train_df = train_df_raw.copy()
test_df = test_df_raw.copy()

In [ ]:
# Convert startdate from object to various usable types. Month of the year has more impact on weather so Year, Month and Day will be separated
train_df['startdate'] = pd.to_datetime(train_df['startdate'], format='%m/%d/%y')
train_df['startdate_ordinal'] = train_df['startdate'].apply(lambda x:x.toordinal())
train_df['year'] = train_df['startdate'].dt.year
train_df['month'] = train_df['startdate'].dt.month
train_df['dayofyear'] = train_df['startdate'].dt.day_of_year

test_df['startdate'] = pd.to_datetime(test_df['startdate'], format='%m/%d/%y')
test_df['startdate_ordinal'] = test_df['startdate'].apply(lambda x:x.toordinal())
test_df['year'] = test_df['startdate'].dt.year
test_df['month'] = test_df['startdate'].dt.month
test_df['dayofyear'] = test_df['startdate'].dt.day_of_year

In [ ]:
# Round to 6 decimal places precision to latitude and longitude for all practical purpose
scale = 6
train_df['lat'] = train_df['lat'].round(scale)
train_df['lon'] = train_df['lon'].round(scale)
test_df['lat'] = test_df['lat'].round(scale)
test_df['lon'] = test_df['lon'].round(scale)

In [ ]:
print('Unique locations in train data ',train_df.groupby(['lat','lon']).ngroup().nunique())
print('Unique locations in test data ',test_df.groupby(['lat','lon']).ngroup().nunique()) 
print('Unique locations in combined data ',pd.concat([train_df,test_df], axis=0).groupby(['lat','lon']).ngroup().nunique())

In [ ]:
# For now Haversine distance will not be used, instead Point geometry data for location will be converted to string for use as classification feature

# Need to combine the latitude and longitude for easier data handling
# 'Single-point' Haversine: Calculates the great circle distance between a point on Earth and the (0, 0) lat-long coordinate

# def single_pt_haversine(lat, lon, degrees=True):
    
#     r = 6371 # Earth's radius (km)

#     # Convert decimal degrees to radians
#     if degrees:
#         lat, lon = map(radians, [lat, lon])

#     # 'Single-point' Haversine formula
#     a = sin(lat/2)**2 + cos(lat) * sin(lon/2)**2
#     d = 2 * r * asin(sqrt(a)) 

#     return d

In [ ]:
# Combine latitude and longitude to generate unique geolocations. Convert to String for later use

# train_df['haversine_distance'] = [single_pt_haversine(x, y) for x, y in zip(train_df.lat, train_df.lon)]
train_df['location'] = [Point(xy) for xy in zip(train_df['lat'], train_df['lon'])] 
train_df['location'] = train_df['location'].apply(lambda x: wkt.dumps(x))

test_df['location'] = [Point(xy) for xy in zip(test_df['lat'], test_df['lon'])] 
test_df['location'] = test_df['location'].apply(lambda x: wkt.dumps(x))

In [ ]:
# Check if data is sorted by new location information
print('Sorted by Location = ', check_sort(train_df,'location'))

In [ ]:
with open('locations.txt', 'w', encoding='utf-8') as f:
    for e in list(train_df['location'].unique()):
        f.write(f'{e}\n')

In [ ]:
# Convert Climate regions to string and dummy numerical data for processing
train_df['climateregions__climateregion'] = train_df['climateregions__climateregion'].astype(str)
train_df['climateregions_num'] = LabelEncoder().fit_transform(train_df['climateregions__climateregion'])

test_df['climateregions__climateregion'] = test_df['climateregions__climateregion'].astype(str)
test_df['climateregions_num'] = LabelEncoder().fit_transform(test_df['climateregions__climateregion'])

In [ ]:
# Impute Null/NaN values
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(train_df[nan_features])
  
train_df[nan_features] = imputer.transform(train_df[nan_features])
print(f'Columns with null vaules in Training data are {train_df.columns[train_df.isnull().any()]}')
print(f'Columns with null vaules in Training data are {train_df.columns[train_df.isna().any()]}')

#### Further analyse and Visualize Data 

##### Feature Selection Algorithms

In [ ]:
# features that are non-numeric
nonum_features = ['location','startdate','startdate_ordinal','climateregions__climateregion','lat','lon','index']

# features that are predictions from other models
all_features = list(train_df.columns)
predict_prefix = ('nmme','cancm','ccsm','cfsv20','gfdl','nasa')

predict_features = []
for f in all_features:
    if f.startswith(predict_prefix):
        predict_features.append(f)
print(predict_features)

In [ ]:
# Univariate Selection to identify strongest relationship to target
features_to_drop = nonum_features + predict_features
features_to_drop.append(target_feature)

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)
y = train_df[target_feature].copy()

selector = SelectKBest(f_regression, k=10) # Top 30 features
selector.fit(X, y)
scores = selector.scores_

bestfeatures = pd.DataFrame({'feature': X.columns, 'score': scores})
bestfeatures = bestfeatures.sort_values(by='score', ascending=False)

display(bestfeatures.head(10))
bestfeatures.plot.bar(x='feature', y='score',figsize=(70, 20))

In [ ]:
# ANOVA F-value to identify important features
features_to_drop = nonum_features + predict_features
features_to_drop.append(target_feature)

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)
y = train_df[target_feature].copy()

selector = SelectKBest(f_classif, k=10) # Top 30 features
selector.fit(X, y)
scores = selector.scores_

bestfeatures = pd.DataFrame({'feature': X.columns, 'score': scores})
bestfeatures = bestfeatures.sort_values(by='score', ascending=False)

display(bestfeatures.head(10))
bestfeatures.plot.bar(x='feature', y='score',figsize=(70, 20))

In [ ]:
# Pearson Correlation to identify strongest relationship to target
features_to_drop = nonum_features + predict_features

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)

y = train_df[target_feature].copy()

corr = X.corr()
# corr = corr[(corr >= 0.8) & (corr != 1.0)].dropna(how='all').dropna(axis=1, how='all')
cols = corr.loc[corr[target_feature] >= 0.7, target_feature].index
corr = corr.loc[cols, cols]
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
# Kendall's method Correlation to identify strongest relationship to target
# It takes a long time to run

features_to_drop = nonum_features + predict_features

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)

y = train_df[target_feature].copy()

corr = X.corr(method='kendall')
# corr = corr[(corr >= 0.8) & (corr != 1.0)].dropna(how='all').dropna(axis=1, how='all')
cols = corr.loc[corr[target_feature] >= 0.7, target_feature].index
corr = corr.loc[cols, cols]
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()

Using NMME and other forecast data for improving prediction

In [ ]:
# Univariate Selection using NMME and other forecast data for improving prediction
features_to_drop = nonum_features
features_to_drop.append(target_feature)

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)
y = train_df[target_feature].copy()

selector = SelectKBest(f_classif, k=10) # Top 30 features
selector.fit(X, y)
scores = selector.scores_

bestfeatures = pd.DataFrame({'feature': X.columns, 'score': scores})
bestfeatures = bestfeatures.sort_values(by='score', ascending=False)

display(bestfeatures.head(10))
bestfeatures.plot.bar(x='feature', y='score',figsize=(70, 20))

In [ ]:
# Recursive Feature Elimination (RFE) method
features_to_drop = nonum_features
features_to_drop.append(target_feature)

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)
y = train_df[target_feature].copy()

reg = LinearRegression()

rfe = RFE(reg, n_features_to_select=30)
rfe.fit(X,y)

bestfeatures = X.columns[rfe.support_]
print("Best features :", bestfeatures)


In [ ]:
# Recursive Feature Elimination (RFE) method without NMME features
features_to_drop = nonum_features + predict_features
features_to_drop.append(target_feature)

X = train_df.copy()
X = X.drop(features_to_drop, axis=1)
y = train_df[target_feature].copy()

reg = LinearRegression()

rfe = RFE(reg, n_features_to_select=10)
rfe.fit(X,y)

bestfeatures = X.columns[rfe.support_]
print("Best features :", bestfeatures)

##### Important features
- Geopotential height
- Evaporation
- Precipitable water for entire atmosphere
- Zonal wind
- Longitudinal wind
- Sea level pressure
- Sea ice concentration
- Month of the year
- Day of the year

Came up in RFE method
- Humidity
- MEI /NIP
- MJO amplitude
- Climate regions
- year?

##### Location Temperature Variation

In [ ]:
temp_df = train_df[['location','startdate_ordinal','contest-tmp2m-14d__tmp2m']].copy()

# Need to convert categorical data to numerical data
temp_df['location']=pd.Categorical(temp_df['location'])
temp_df['location']=temp_df['location'].cat.codes

fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(temp_df['location'], temp_df['startdate_ordinal'],  temp_df['contest-tmp2m-14d__tmp2m'], c=temp_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

##### Yearly, Monthly, Daily Temperature Variation

In [ ]:
#Yearly variation
temp_df = train_df[['year','contest-tmp2m-14d__tmp2m']].copy()
temp_df.plot.scatter(x='year',y='contest-tmp2m-14d__tmp2m', c=temp_df[target_feature], cmap=plt.cm.jet)

Temperature has risen by at least a couple of degrees as per yearly variation

In [ ]:
#Monthly variation
temp_df = train_df[['month','contest-tmp2m-14d__tmp2m']].copy()
temp_df.plot.scatter(x='month',y='contest-tmp2m-14d__tmp2m', c=temp_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Daily variation
temp_df = train_df[['dayofyear','contest-tmp2m-14d__tmp2m']].copy()
temp_df.plot.scatter(x='dayofyear',y='contest-tmp2m-14d__tmp2m', c=temp_df[target_feature], cmap=plt.cm.jet)

##### Elevation

In [ ]:
# Confirm if elevation is consistant for a location
# elevation_df = train_df[['location','elevation__elevation']].drop_duplicates().copy()
print('Unique combination of location and elevation are ',train_df[['location','elevation__elevation']].drop_duplicates().shape[0])

In [ ]:
#Elevation variation
elevation_df = train_df[['elevation__elevation','contest-tmp2m-14d__tmp2m']].copy()
elevation_df.plot.scatter(x='elevation__elevation',y='contest-tmp2m-14d__tmp2m', c=elevation_df[target_feature], cmap=plt.cm.jet)

The Effect of elevation is conjunction with latitude but overall understanding is that higher altitude leads to lower temperatures

In [ ]:
# Elevation Temperature variation over time
elevation_df = train_df[['elevation__elevation','startdate','contest-tmp2m-14d__tmp2m']].copy()

fig = plt.figure(dpi=300)
ax = plt.axes(projection='3d')
ax.plot_trisurf(elevation_df['elevation__elevation'], elevation_df['startdate'],  elevation_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

In [ ]:
# Visualize temperature against elevation over time
# elevation_df = train_df[['elevation__elevation','startdate','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

elevations = elevation_df.elevation__elevation.unique()
elevations.sort()

# Plot temperature against elevation for a group of elevations 
# for e in itertools.islice(elevations, 40, 43):
#     elevation_df_1 = elevation_df[elevation_df['elevation__elevation']==e]
#     print('Elevation ',e)
#     elevation_df_1.plot.line(x='startdate',y='contest-tmp2m-14d__tmp2m')

# Plot temperature against elevation for a range of elevations
for e in list(filter(lambda e: (e>=100 and e<=300), elevations)):
    elevation_df_1 = elevation_df[elevation_df['elevation__elevation'] == e]
    title_str = 'Elevation' + str(e)
    elevation_df_1.plot.line(x='startdate',y='contest-tmp2m-14d__tmp2m',title=title_str)

The effect of temperature with changing elevation is clear. Therefore this is an important feature.

##### Geopotential height

In [ ]:
# Geopotential height Temperature variation
geoheight = 'contest-wind-h10-14d__wind-hgt-10'
elevation_df = train_df[[geoheight,'contest-tmp2m-14d__tmp2m']].copy()
elevation_df.plot.scatter(x=geoheight,y='contest-tmp2m-14d__tmp2m', c=elevation_df[target_feature], cmap=plt.cm.jet)

Geopotential height is a much more reliable feature than elevation

In [ ]:
# Geopotential height Temperature variation over time
geoheight = 'contest-wind-h10-14d__wind-hgt-10'
elevation_df = train_df[[geoheight,'startdate','contest-tmp2m-14d__tmp2m']].copy()

fig = plt.figure(dpi=300)
ax = plt.axes(projection='3d')
ax.plot_trisurf(elevation_df[geoheight], elevation_df['startdate'],  elevation_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

##### Evaporation

In [ ]:
# Evaporation Temperature variation
evaporation = 'contest-pevpr-sfc-gauss-14d__pevpr'
eva_df = train_df[[evaporation,target_feature]].copy()
eva_df.plot.scatter(x=evaporation,y=target_feature, c=eva_df[target_feature], cmap=plt.cm.jet)

##### Precipitable water

In [ ]:
# Precipitable water Temperature variation
prwtr_df = train_df[['contest-prwtr-eatm-14d__prwtr',target_feature]].copy()
prwtr_df.plot.scatter(x='contest-prwtr-eatm-14d__prwtr',y=target_feature, c=prwtr_df[target_feature], cmap=plt.cm.jet)

##### Precipitation

In [ ]:
# Precipitation Temperature variation
prec_df = train_df[['contest-precip-14d__precip',target_feature]].copy()
prec_df.plot.scatter(x='contest-precip-14d__precip',y=target_feature, c=prec_df[target_feature], cmap=plt.cm.jet)

##### Relative Humidity

In [ ]:
# Relative Humidity Temperature variation
hum_df = train_df[['contest-precip-14d__precip',target_feature]].copy()
hum_df.plot.scatter(x='contest-precip-14d__precip',y=target_feature, c=hum_df[target_feature], cmap=plt.cm.jet)

##### Zonal wind

In [ ]:
# Zonal wind Temperature variation
uwind = 'contest-wind-uwnd-925-14d__wind-uwnd-925'
wind_df = train_df[[uwind,target_feature]].copy()
wind_df.plot.scatter(x=uwind,y=target_feature, c=wind_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Zonal wind Temperature variation over location
uwind = 'contest-wind-uwnd-925-14d__wind-uwnd-925'
wind_df = train_df[[uwind,'location','contest-tmp2m-14d__tmp2m']].copy()


# Need to convert categorical data to numerical data
wind_df['location']=pd.Categorical(wind_df['location'])
wind_df['location']=wind_df['location'].cat.codes

fig = plt.figure(dpi=300)
ax = plt.axes(projection='3d')
ax.plot_trisurf(wind_df[uwind], wind_df['location'], wind_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

There is a tendency of decrease in temperature due to higher zonal wind speed

##### Longitudinal Wind

In [ ]:
# Longitudinal wind Temperature variation
vwind = 'contest-wind-vwnd-250-14d__wind-vwnd-250'
wind_df = train_df[[vwind,target_feature]].copy()
wind_df.plot.scatter(x=vwind,y=target_feature, c=wind_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Longitudinal wind Temperature variation over location
vwind = 'contest-wind-vwnd-250-14d__wind-vwnd-250'
wind_df = train_df[[vwind,'location','contest-tmp2m-14d__tmp2m']].copy()


# Need to convert categorical data to numerical data
wind_df['location']=pd.Categorical(wind_df['location'])
wind_df['location']=wind_df['location'].cat.codes

fig = plt.figure(dpi=300)
ax = plt.axes(projection='3d')
ax.plot_trisurf(wind_df[vwind], wind_df['location'], wind_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

There is a tendency of rise in temperature with lower wind speed. Negative direction is more cooler

##### Climate Region

In [ ]:
# Climate region Temperature variation
climate_df = train_df[['climateregions__climateregion','contest-tmp2m-14d__tmp2m']].copy()
climate_df.plot.scatter(x='climateregions__climateregion',y='contest-tmp2m-14d__tmp2m', c=elevation_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Climate region temperature variation - monthly
climate_df = train_df[['climateregions__climateregion','month','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

# Need to convert categorical data to numerical data
climate_df['climateregions__climateregion']=pd.Categorical(climate_df['climateregions__climateregion'])
climate_df['climateregions__climateregion']=climate_df['climateregions__climateregion'].cat.codes

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')
ax.scatter3D(climate_df['climateregions__climateregion'], climate_df['month'],  climate_df['contest-tmp2m-14d__tmp2m'], c=climate_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

In [ ]:
# Plot effect of climate region on temperature over time
climate_df = train_df[['climateregions__climateregion','startdate','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

climates = climate_df.climateregions__climateregion.unique()
display(print('Unique climate regions ',len(climates)))

for e in climates:
    climate_df_1 = climate_df[climate_df['climateregions__climateregion'] == e]
    title_str = 'Climate region ' + e
    climate_df_1.plot.line(x='startdate',y='contest-tmp2m-14d__tmp2m',title=title_str)

In [ ]:
# Combined plot using plotly express

fig = px.line(climate_df, x='startdate', 
              y='contest-tmp2m-14d__tmp2m', 
              color = 'climateregions__climateregion', 
              facet_row='climateregions__climateregion',facet_row_spacing=0.04,
              labels={"contest-tmp2m-14d__tmp2m":"Temp", "climateregions__climateregion":"Climate Region"},
              template = 'plotly_white', height=2000)

fig.update_layout(title='Mean temperature variations by climate regions', xaxis_title='Date')
fig.update_yaxes(visible=True, matches=None)
fig.update_layout(annotations=[], overwrite=True)

fig.show()

The effect of climate region on temperature is evident from the plots. But the effect has lower correlation than other factors. Therefore this is a semi important feature.

##### Pressure and Sea Level Pressure

In [ ]:
# Atmospheric pressure Temperature variation
prs_df = train_df[['contest-pres-sfc-gauss-14d__pres',target_feature]].copy()
prs_df.plot.scatter(x='contest-pres-sfc-gauss-14d__pres',y=target_feature, c=prs_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Sea level pressure Temperature variation
prs_df = train_df[['contest-slp-14d__slp',target_feature]].copy()
prs_df.plot.scatter(x='contest-slp-14d__slp',y=target_feature, c=prs_df[target_feature], cmap=plt.cm.jet)

Atmospheric pressure and Sea level pressure are directly proportional to temperature, but the data doesn't show that!!

#####  Multivariate ENSO index

In [ ]:
# Visualize effect of El Niño on temperature
print('Unique combination of location and NIP are ',train_df[['location','mei__nip']].drop_duplicates().shape[0])
print('Unique combination of location and MEI are ',train_df[['location','mei__mei']].drop_duplicates().shape[0])

In [ ]:
# Visualize effect of El Niño on temperature
mei_df = train_df[['mei__meirank','contest-tmp2m-14d__tmp2m']].copy()
mei_df.plot.scatter(x='mei__meirank',y='contest-tmp2m-14d__tmp2m', c=elevation_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Visualize effect of El Niño on temperature over time
mei_df = train_df[['mei__mei','startdate_ordinal','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

meis = mei_df.mei__mei.unique()
meis.sort()

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')
ax.scatter3D(mei_df['mei__mei'], mei_df['startdate_ordinal'],  mei_df['contest-tmp2m-14d__tmp2m'], c=mei_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

In [ ]:
# Visualize temperature against MEI
mei_df = train_df[['mei__mei','startdate','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

meis = mei_df.mei__mei.unique()
meis.sort()

# Plot temperature against elevation for a range of MEI
for e in list(filter(lambda e: (e>=0 and e<=0.5), meis)):
    mei_df_1 = mei_df[mei_df['mei__mei'] == e]
    title_str = 'MEI ' + str(e)
    mei_df_1.plot.line(x='startdate',y='contest-tmp2m-14d__tmp2m',title=title_str)


In [ ]:
# Visualize temperature against Nino Index Phase
nip_df = train_df[['mei__nip','startdate','contest-tmp2m-14d__tmp2m']].copy()

fig = px.line(nip_df, x='startdate', 
              y='contest-tmp2m-14d__tmp2m', 
              color = 'mei__nip', 
              facet_row='mei__nip',facet_row_spacing=0.04,
              labels={"contest-tmp2m-14d__tmp2m":"Temp", "mei__nip":"NIP"},
              template = 'plotly_white', height=300)

fig.update_layout(title='Mean temperature variations by NIP', xaxis_title='Date')
fig.update_yaxes(visible=True, matches=None)
fig.update_layout(annotations=[], overwrite=True)

fig.show()

Effect of MEI and NIP is not clearly visible, although slight increase in temperature is evident with nigher NIP. This is a secondary feature.

##### Madden-Julian Oscillation

In [ ]:
# Visualize effect of MJO on temperature
print('Unique combination of location and MJO Amplitude are ',train_df[['location','mjo1d__amplitude']].drop_duplicates().shape[0])
print('Unique combination of location and MJO Phase are ',train_df[['location','mjo1d__phase']].drop_duplicates().shape[0])

In [ ]:
# Visualize effect of MJO on temperature
mjo_df = train_df[['mjo1d__phase','contest-tmp2m-14d__tmp2m']].copy()
mjo_df.plot.scatter(x='mjo1d__phase',y='contest-tmp2m-14d__tmp2m', c=elevation_df[target_feature], cmap=plt.cm.jet)

In [ ]:
# Visualize effect of MJO on temperature over time
mjo_df = train_df[['mjo1d__phase','startdate_ordinal','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

meis = mjo_df.mjo1d__phase.unique()
meis.sort()

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')
ax.scatter3D(mjo_df['mjo1d__phase'], mjo_df['startdate_ordinal'],  mjo_df['contest-tmp2m-14d__tmp2m'], c=mjo_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

In [ ]:
# Visualize temperature against MJO Phase
mjo_df = train_df[['mjo1d__phase','startdate','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

mjos = mjo_df.mjo1d__phase.unique()
mjos.sort()

# Plot temperature against elevation for a range of MEI
# for e in list(filter(lambda e: (e>=3 and e<=7), mjos)):
#     mjo_df_1 = mjo_df[mjo_df['mjo1d__phase'] == e]
#     title_str = 'MJO Phase ' + str(e)
#     mjo_df_1.plot.line(x='startdate',y='contest-tmp2m-14d__tmp2m',title=title_str)

fig = px.line(mjo_df, x='startdate', 
              y='contest-tmp2m-14d__tmp2m', 
              color = 'mjo1d__phase', 
              facet_row='mjo1d__phase',facet_row_spacing=0.04,
              labels={"contest-tmp2m-14d__tmp2m":"Temp", "mjo1d__phase":"MJO"},
              template = 'plotly_white', height=2000)

fig.update_layout(title='Mean temperature variations by MJO Phase', xaxis_title='Date')
fig.update_yaxes(visible=True, matches=None)
fig.update_layout(annotations=[], overwrite=True)

fig.show()

Effect of MJO is not clearly visible, although slight increase in temperature is evident with nigher MJO phase. This is a secondary feature.

##### Sea Surface Temperature

In [ ]:
# Visualize effect of Sea Surface Temperature on temperature
print('Unique combination of location and Sea Surface Temperature are ',train_df[['location','sst-2010-8']].drop_duplicates().shape[0])

In [ ]:
# Visualize temperature against Sea Surface Temperature
sst = 'sst-2010-1'
sst_df = train_df[[sst,'startdate_ordinal','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

ssts = sst_df[sst].unique()
ssts.sort()

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')
ax.scatter3D(sst_df[sst], sst_df['startdate_ordinal'],  sst_df['contest-tmp2m-14d__tmp2m'], c=sst_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

In [ ]:
# Visualize temperature against Sea Surface Temperature
loc = 'POINT (0.6818180000000000 0.8333330000000000)' #checking by a location
sst = 'sst-2010-1'

sst_df = train_df[['location',sst,'startdate_ordinal','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()
sst_df = sst_df[sst_df['location']==loc]

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')
ax.scatter3D(sst_df[sst], sst_df['startdate_ordinal'],  sst_df['contest-tmp2m-14d__tmp2m'], c=sst_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()


Increase in the Sea Surface Temperature causes increase in Temperature. This is an important feature

##### Sea Ice Concentration

In [ ]:
# Visualize temperature against Sea Ice Concentration
icec = 'icec-2010-1'
icec_df = train_df[[icec,'startdate_ordinal','contest-tmp2m-14d__tmp2m']].drop_duplicates().copy()

icecs = icec_df[icec].unique()
icecs.sort()

fig = plt.figure(dpi=200)
ax = plt.axes(projection='3d')
ax.scatter3D(icec_df[icec], icec_df['startdate_ordinal'],  icec_df['contest-tmp2m-14d__tmp2m'], c=icec_df['contest-tmp2m-14d__tmp2m'], cmap=plt.cm.jet, linewidth=0.01)
plt.show()

##### NMME prediction comparison

In [ ]:
# NMME Temperature comparison with actual
loc = 'POINT (1.0000000000000000 0.6000000000000000)' #checking by a location
nmme_temp = 'nmme0mean'

temp_df = train_df[['location','startdate',nmme_temp,target_feature]].copy()
temp_df = temp_df[temp_df['location']==loc]
temp_df['difference'] = temp_df[nmme_temp]-temp_df[target_feature]

temp_df.plot.bar(x='startdate',y='difference')
# temp_df.plot.scatter(x='startdate_ordinal',y='difference')

The NMME predictions are off to various degrees in both positive and negative direction. This poses a question on whether this data should be used.

##### Additional important features
- Elevation
- Sea Surface Temperature
- Climate Region

#### Inferences
- NMME forecast data have the strongest correlation to the target as per various feature selection algorithms
- The algorithms are contradictory about feature selection in some cases!
- The common weather forecast parameters are not considered by the algorithms!
- If individually compared the data sometimes doesn't correlate strongly where ideally it should as per meteorological equations!
- There are too many features in the dataset and choosing features for modelling is a challenging task
- Any of the following strategies may be followed - 
    - Select the results of the feature selection algorithms which puts heavy preference on NMME forecast data
    - Select known meteorological parameters which affect temperature and ignore NMME forecast data
    - Combine from both feature selection algorithm results and meteorological parameters
- Next question to answer how many features to be considered?
- Many features have multiple measurements from different methods. Should all be considered or only one or select few?

## Data Preparation

## Modelling

#### Multiple Linear Regression

#### Gradient Descent Optimization

#### Non-Linear Regression

#### K-Nearest-Neighbour algorithm

#### K Means Clustering Algorithm

#### Artificial Neural Networks

## Evaluation